In [1]:
from langchain_core.output_parsers import StrOutputParser,ListOutputParser,JsonOutputParser,PydanticOutputParser

In [57]:
from operator import itemgetter
from langchain_groq import ChatGroq
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.pydantic_v1 import BaseModel, Field,validator
import os
from bs4 import BeautifulSoup as Soup
from langchain_core.runnables import RunnableLambda


In [4]:
llm = ChatGroq(api_key=os.getenv("GROQ_API_KEY"),model='mixtral-8x7B-32768')

In [5]:
def extract_text(x):
    return Soup(x,'html.parser').text

In [13]:
url="https://developer.nytimes.com/docs/books-product/1/overview"
loader=RecursiveUrlLoader(
    url=url,
    max_depth=20,
    extractor=extract_text
)

In [14]:
docs=loader.load()


In [25]:
docs_sorted=sorted(docs,key=lambda x:x.metadata["source"])
docs_reversed=list(reversed(docs_sorted))



In [28]:
docs[0].metadata['source']

'https://neuro.georgetown.edu/about-neuroscience/'

In [29]:
#Concatenate page_content of each sorted dictionary
concatenated_content="\n\n\n----\n\n\n".join([doc.page_content for doc in docs_reversed])



In [32]:
from langchain_core.prompts.prompt import PromptTemplate

In [52]:

class code(BaseModel):
    """ Code Output """
    answer: str = Field(description='Returned answers')


parser=PydanticOutputParser(pydantic_object=code)
template="""
You are an expert reader who provides answers after reading some content.\n
{format_instructions}
Here is the content:
\n---------\n
{context}
\n---------\n
Answer the questions based on above content.\n
Here is the user question :\n
--- --- --- --- \n
{question}
"""

In [61]:
prompt=PromptTemplate(
    template=template,
    input_variables=["context","question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
chain={
        "context":RunnableLambda(lambda x:concatenated_content), #should be runnables
        "question":RunnableLambda(itemgetter("question")),
    }|prompt|llm |parser


response = chain.invoke({
    "question":"what is neuroinformatics"
})